In [ ]:

# FastAPI Spam Detector Demo 

import nest_asyncio
import uvicorn
import joblib
from fastapi import FastAPI
from pydantic import BaseModel
import threading
import webbrowser
from pathlib import Path
import time 
nest_asyncio.apply()  # allows uvicorn to run inside Jupyter


# 1. Load trained pipeline (from 02_modeling.ipynb output)

MODEL_PATH = Path("../notebooks/experiments/model_lr_word.joblib")

if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}. Run 02_modeling.ipynb first.")

pipe = joblib.load(MODEL_PATH)



In [5]:

# 2. Request / Response Schemas

class EmailIn(BaseModel):
    text: str

class PredictionOut(BaseModel):
    label: str
    probability_spam: float | None = None


# 3. Prediction helper

def predict_text(text: str) -> dict:
    """
    Input: single string
    Output: dict with spam_or_not and probability_spam
    """
    # The pipeline expects a list of samples
    input_list = [text]
    
    # Predict label
    pred = pipe.predict(input_list)[0]
    label = "spam" if pred == 1 else "ham"

    # Predict probability if available
    prob = None
    if hasattr(pipe, "predict_proba"):
        prob = float(pipe.predict_proba(input_list)[0][1])
    
    return {"spam_or_not": label, "probability_spam": prob}



In [6]:

# 4. Define request & response schemas

from fastapi import FastAPI
from pydantic import BaseModel

class EmailIn(BaseModel):
    text: str

class PredictionOut(BaseModel):
    spam_or_not: str
    probability_spam: float | None = None

app = FastAPI(title="Spam Detector API", version="1.0")

@app.post("/predict", response_model=PredictionOut)
def predict(email: EmailIn):
    return predict_text(email.text)

# 5. Prediction function

def predict_text(text: str):
    """Return {'spam_or_not': 'spam'|'ham', 'probability_spam': float|None}"""
    # Always pass the text as a 1-element list
    pred = pipe.predict([text])[0]
    label = "spam" if pred == 1 else "ham"

    probability_spam = None
    clf = pipe.named_steps["clf"]
    if hasattr(clf, "predict_proba"):
        probability_spam = float(pipe.predict_proba([text])[0][1])

    return {"spam_or_not": label, "probability_spam": probability_spam}





In [ ]:
# 4. FastAPI app and routes

app = FastAPI(title="Spam Detector API", version="1.0")

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/predict", response_model=PredictionOut)
def predict(email: EmailIn):
    return predict_text(email.text)


# 5. Run ASGI server inside a notebook

from uvicorn import Config, Server

def open_docs():
    time.sleep(1)  # wait a bit for server to start
    webbrowser.open("http://127.0.0.1:8001/docs")
threading.Thread(target=open_docs).start()

config = Config(app=app, host="127.0.0.1", port=8001, loop="asyncio")
server = Server(config=config)

threading.Thread(target=lambda: server.run()).start()

await server.serve()

INFO:     Started server process [443074]
INFO:     Started server process [443074]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.
INFO:     ASGI 'lifespan' protocol appears unsupported.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8001): address already in use


INFO:     127.0.0.1:41836 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:41836 - "GET /openapi.json HTTP/1.1" 200 OK
